In [ ]:
import os
import json
import pandas as pd
from datetime import datetime

In [ ]:
# time_name = 'published_at'
time_name = 'time'
def extract_time_difference_from_subgraph(subgraph):
    nodes = subgraph['nodes']

    for node in nodes:
        if node[time_name] != "Not available":
            node[time_name] = datetime.fromisoformat(node[time_name][:-1])
        else:
            node[time_name] = None

    dates = [node[time_name] for node in nodes if node[time_name] is not None]
    if dates:
        min_date = min(dates)
        max_date = max(dates)
        time_diff = max_date - min_date  

        time_diff_in_days = time_diff.total_seconds() / 86400  # 86400s = 1d
        time_diff_str = f"{time_diff_in_days:.2f}"
    else:
        time_diff_str = 'None'
    
    return time_diff_str

def process_json_files_in_folders(folders):
    time_differences = []

    for folder_path in folders:
        for root, _, files in os.walk(folder_path):
            for filename in files:
                if filename.endswith('.json'):
                    file_path = os.path.join(root, filename)
                    with open(file_path, 'r') as file:
                        subgraph = json.load(file)
                        time_diff_str = extract_time_difference_from_subgraph(subgraph)
                        
                        time_differences.append({
                            'file_path': file_path,
                            'time_difference(Day)': time_diff_str
                        })

    df = pd.DataFrame(time_differences)
    output_path = './time_differences_source.xlsx'
    df.to_excel(output_path, index=False)
    print(f"Time differences saved to {output_path}")

folders = [
  'KG/Co-existing_Edge/source_report/npm',
  'KG/Co-existing_Edge/source_report/pypi',
  'KG/Co-existing_Edge/source_report/ruby'
]
process_json_files_in_folders(folders)


In [ ]:
def process_excel(input_excel_path, output_excel_path):
    df = pd.read_excel(input_excel_path)

    # Create a list to store the max time difference for each cluster
    result = []

    # Filter out rows where 'published_at' is 'Not available'
    df = df[df['published_at'] != 'Not available']

    # Ensure the 'published_at' column is in datetime format
    df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')

    # Iterate through each cluster group
    for cluster, group in df.groupby('cluster'):
        # Sort the group by published_at
        group_sorted = group.sort_values(by='published_at')
        
        if len(group_sorted) > 1:
            time_diff = group_sorted['published_at'].max() - group_sorted['published_at'].min()
            # Convert time difference to days, keeping decimals
            time_diff_in_days = time_diff.total_seconds() / 86400  # 86400 seconds = 1 day
            time_diff_str = f"{time_diff_in_days:.2f}"  # Round to 2 decimal places
            result.append(["ruby_" + str(cluster), time_diff_str])
        else:
            result.append(["ruby_" + str(cluster), "0.00"])

    result_df = pd.DataFrame(result, columns=['cluster', 'time'])

    result_df.to_excel(output_excel_path, index=False)

    print(f"Results have been saved to {output_excel_path}")

if __name__ == "__main__":
    input_excel_path = 'KG/Similar_Edge/ruby_end.xlsx'  # Path to input Excel file
    output_excel_path = './ruby_active.xlsx'  # Path to output Excel file
    process_excel(input_excel_path, output_excel_path)